<a href="https://colab.research.google.com/github/jjung2-oxy/Natural-Lang-Processing/blob/main/NLP_Homework_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**TFIDF Search Engine**:
- To search for a user query, this code compares the average tfidf cosine similarity score of the query and the title, description, and transcript.
- For search results that have close average tfidf cosine scores, we 'tiebreak' them by looking for matching named entities and nouns in the related topics and speakers categories. Lastly we give extra points to results that have highest number of views compared to other results with similar tfidf cosine scores.
- To use, run all the functions before executing the last two blocks. The first of the last two will get the csv uploaded and do the preprocessing, this should take at most 5 minutes or so. The last cell asks for the search query and finds matching results to print. This will take about 30 seconds.
- Improvements: with more time, we would've added extra points for more recently dated talks, tried bm25 instead of tfidf, looked at comments and duration for extra points, and made the searching run faster. As of now, each search is about 30 seconds because the looking for extra points for each results takes a bit of time. When a search query can't return a full 10 results, it prints out "Other results you may be interested in" by taking them from the related talks of the top result. This is not very advanced and so improving that would be awesome. What we were wishing we could've done is if a specific search didn't return anything, split the query apart and look at the words individually or do a 'did you mean...' search similar to how Google does it.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import os
import json
import ast
from nltk import word_tokenize
from gensim.summarization.bm25 import BM25
from gensim import corpora
import re
from termcolor import colored
from tqdm import tqdm as tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.tag import pos_tag
from nltk.chunk import conlltags2tree, tree2conlltags
from pprint import pprint
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
from nltk.corpus import stopwords
stopwords = nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download("punkt")
pd.set_option('display.max_colwidth', None)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
def preprocess(documents, remove_sw=True, stem=True, untokenized=True, verbose=True):
    '''
    Preprocesses documents by case folding, removing punctuation, tokenizing, stopword removal, stemming, and untokenizing before returning.

    Inputs:
    - documents : list of tuples of (text_item, ted_talk_id).
    - remove_sw : toggles removal of stopwords
    - stem : toggles stemming of tokens
    - verbose: toggles progress bar for preprocessing (tqdm)

    Output:
    - processed_docs : list of tuples of (processed_text_item, ted_talk_id)

    '''

    verboseprint = print if verbose else lambda *a, **k: None

    processed_docs = []

    # Preprocess documents
    for doc, talk_id in tqdm(documents, total=len(documents), disable = not verbose):
        # Case folding
        doc = doc.lower();
      
        # Remove punctuation
        import string
        doc = "".join([char for char in doc if char not in string.punctuation])

        # Tokenization for stopword removal and stemming
        from nltk import word_tokenize
        processed = word_tokenize(doc)

        # Remove stopwords
        if remove_sw == True:
            stopwords = nltk.corpus.stopwords.words("english")
            processed = [word for word in processed if word not in stopwords]
 
        # Stemming
        from nltk.stem.porter import PorterStemmer
        if stem == True:
            porter = PorterStemmer()
            processed = [porter.stem(word) for word in processed]

        # Untokenize for TFIDF fitting.
        if untokenized == True:
            processed = ' '.join(word for word in processed)

        # Append processed text as (text, talk_id)
        processed_docs.append((processed, talk_id))

    verboseprint(f"Processed documents: {processed_docs}")

    return processed_docs


def preprocess_no_id(documents, remove_sw=True, stem=True, untokenized=True, verbose=True):
    '''
    Preprocesses documents by case folding, removing punctuation, tokenizing, stopword removal, stemming, and untokenizing before returning.

    Inputs:
    - documents : list of documents
    - remove_sw : toggles removal of stopwords
    - stem : toggles stemming of tokens
    - verbose: toggles progress bar for preprocessing (tqdm)

    Output:
    - processed_docs : list of processed documents

    '''

    verboseprint = print if verbose else lambda *a, **k: None

    processed_documents = []

    if type(documents) == list:
        # Preprocess documents
        for doc in tqdm(documents, total=len(documents), disable = not verbose):
            # Case folding
            doc = doc.lower();
          
            # Remove punctuation
            import string
            doc = "".join([char for char in doc if char not in string.punctuation])

            # Tokenization for stopword removal and stemming
            from nltk import word_tokenize
            processed = word_tokenize(doc)

            # Remove stopwords
            if remove_sw == True:
                stopwords = nltk.corpus.stopwords.words("english")
                processed = [word for word in processed if word not in stopwords]
    
            # Stemming
            from nltk.stem.porter import PorterStemmer
            if stem == True:
                porter = PorterStemmer()
                processed = [porter.stem(word) for word in processed]

            # Untokenize for TFIDF fitting.
            if untokenized == True:
                processed = ' '.join(word for word in processed)

            # Append processed text as (text, talk_id)
            processed_documents.append(processed)
        verboseprint(f"Processed Documents: {processed_documents}")
        return processed_documents

    else:
        doc = documents
        # Case folding
        doc = doc.lower();
          
        # Remove punctuation
        import string
        doc = "".join([char for char in doc if char not in string.punctuation])

        # Tokenization for stopword removal and stemming
        from nltk import word_tokenize
        processed = word_tokenize(doc)

        # Remove stopwords
        if remove_sw == True:
            stopwords = nltk.corpus.stopwords.words("english")
            processed = [word for word in processed if word not in stopwords]
    
        # Stemming
        from nltk.stem.porter import PorterStemmer
        if stem == True:
            porter = PorterStemmer()
            processed = [porter.stem(word) for word in processed]

        # Untokenize for TFIDF fitting.
        if untokenized == True:
            processed = ' '.join(word for word in processed)

        # Append processed text as (text, talk_id)
        verboseprint(f"Single doc: {processed}")
        return processed

In [ ]:
def tfidf_fit(documents, top_n_terms=10):
    '''
    Creates a tfidf model and fits it with given text documents.

    Inputs:
    - documents : list of text documents
    - top_n_terms : number of top terms to return

    Outputs:
    - tfidf_vect : TfIdfVectorizer model object
    - df_tfidfvect : Dataframe of tfidf vectorized document tokens (index = tokens)
    - tfidf_feature_names : list of top_n_terms with highest tfidf importance

    '''


    tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}')
    doc_matrix = tfidf_vect.fit_transform(documents)
    data = doc_matrix.T.toarray()
    tfidf_tokens = tfidf_vect.get_feature_names_out()
    df_tfidfvect = pd.DataFrame(data, index=tfidf_tokens)

    importance = np.argsort(np.asarray(doc_matrix.sum(axis=0)).ravel())[::-1]
    tfidf_feature_names = np.array(tfidf_tokens)

    return tfidf_vect, df_tfidfvect, tfidf_feature_names[importance[:top_n_terms]]



In [ ]:
def main_tfidf(ted_talks_df, verbose=False):
    '''
    Main driver function for preprocessing and tfidf vectorizing Ted Talk title, description, and transcript documents

    Inputs:
    - ted_talks_df : Pandas dataframe of ted_talks csv
    - verbose : print if true, else do nothing (used with verboseprint)

    Outputs (used along with get_top_matches so returns a lot of outputs):
    - tfidf_title_model : tfidf vectorizer model object for Title documents
    - tfidf_title_df : Pandas dataframe of TFIDF vectorized Title tokens (index = title tokens)
    - processed_titles : list of lists with (preprocessed Title text, ted_talk_id)
    - tfidf_description_model : tfidf vectorizer model object for Description documents
    - tfidf_description_df : Pandas dataframe of TFIDF vectorized Description tokens (index = description tokens)
    - processed_description : list of lists with (preprocessed description text, ted_talk_id)
    - tfidf_transcript_model : tfidf vectorizer model object for Transcript documents
    - tfidf_transcript_df : Pandas dataframe of TFIDF vectorized Description tokens (index = transcript tokens)
    - processed_transcripts : list of lists with (preprocessed transcript text, ted_talk_id)

    '''
    verboseprint = print if verbose else lambda *a, **k: None

    # Data Preprocessing for Titles
    title_docs = []
    for index, row in ted_talks_df.iterrows():
        title_docs.append((row["title"], row["talk_id"]))

    print("Processing Title Documents")
    processed_titles = preprocess(title_docs, remove_sw=False, stem = True, verbose=verbose)
    processed_title_docs, title_talk_ids = map(list, zip(*processed_titles))
    verboseprint(f"Length of processed title documents: {len(processed_title_docs)}")
    verboseprint("Example:")
    verboseprint(processed_title_docs[0])
    verboseprint("Generating TfIdf for Titles")
    tfidf_title_model, tfidf_title_df, tfidf_title_topterms = tfidf_fit(processed_title_docs)
    verboseprint(f"Dataframe shape: {tfidf_title_df.shape}")
    verboseprint(f"Top Terms (stopwords included): {tfidf_title_topterms}")
    verboseprint("-----------------------------------------------------\n")

    # Data Preprocessing for Descriptions
    description_docs = []
    for index, row in ted_talks_df.iterrows():
        description_docs.append((row["description"], row["talk_id"]))

    print("Processing Description Documents")
    processed_description = preprocess(description_docs, remove_sw=True, stem = True, verbose=verbose)
    processed_description_docs, description_talk_ids = map(list, zip(*processed_description))
    verboseprint(f"Length of processed description documents: {len(processed_description_docs)}")
    verboseprint("Example:")
    verboseprint(processed_description_docs[0])
    verboseprint("Generating TfIdf for Descriptions")
    tfidf_description_model, tfidf_description_df, tfidf_description_topterms = tfidf_fit(processed_description_docs)
    verboseprint(f"Dataframe shape: {tfidf_description_df.shape}")
    verboseprint(f"Top Terms (stopwords removed): {tfidf_description_topterms}")
    verboseprint("-----------------------------------------------------\n")

    # Data Preprocessing for Transcripts
    transcript_docs = []
    for index, row in ted_talks_df.iterrows():
        transcript_docs.append((row["transcript"], row["talk_id"]))

    print("Processing Transcript Documents")
    processed_transcripts = preprocess(transcript_docs, verbose=verbose)
    processed_transcript_docs, transcript_talk_ids = map(list, zip(*processed_transcripts))
    verboseprint(f"\nLength of processed transcript documents: {len(processed_transcript_docs)}")
    verboseprint("Example:")
    verboseprint(processed_transcript_docs[0])
    verboseprint("Generating TfIdf for Transcripts")
    tfidf_transcript_model, tfidf_transcript_df, tfidf_transcript_topterms = tfidf_fit(processed_transcript_docs)
    verboseprint(f"Dataframe shape: {tfidf_transcript_df.shape}")
    verboseprint(f"Top Terms (stopwords removed): {tfidf_transcript_topterms}")
    verboseprint("-----------------------------------------------------\n")

    return tfidf_title_model, tfidf_title_df, processed_titles, tfidf_description_model, tfidf_description_df, processed_description, tfidf_transcript_model, tfidf_transcript_df, processed_transcripts

In [ ]:
def process_query(query, tfidf_model, tfidf_df, title_search=False, verbose=False):
    '''
    Preprocesses query, and vectorizes with given tfidf model object

    Inputs:
    - query : user query (str)
    - tfidf_model : tfidf model fitted with given documents (tfidf_model object)
    - document_df : Pandas Dataframe of tfidf matrix for given documents
    - title_search : whether to run preprocess with stopwords or not. For title search, do not remove stopwords.
    - verbose : whether to print outputs (with verboseprint)

    Outputs:
    - q_veb : tfidf vectorized query

    '''

    verboseprint = print if verbose else lambda *a, **k: None
    
    verboseprint("Query:", query)
    if title_search == True:
        processed_query = preprocess_no_id(user_query, remove_sw=False, stem=True, verbose=False)
    else:
        processed_query = preprocess_no_id(user_query, remove_sw=True, stem=True, verbose=False)
    
    verboseprint("\nProcessed Query:", processed_query + "\n")

    # Convert the query to a vector
    q = [processed_query]
    q_vec = tfidf_model.transform(q).toarray().reshape(tfidf_df.shape[0],)

    return q_vec

def get_similar_documents(query_vector, tfidf_model, df, docs, verbose=False):
    '''
    Find cosine similarity of query vector and every document in the given column.

    Inputs:
    - query_vector : tfidf vectorized user query
    - tfidf_model : TfidfVectorizer() model object
    - df : Pandas dataframe of tfidf matrix (index = document tokens)
    - docs : list of tuples with (processed text, talk_id)
    - verbose : whether to print outputs (with verboseprint)

    Outputs:
    - top_sorted : documents with cosine similarity scores >= 0.3 
    - perfect_match : document with cosine similarity score of 1.0 (exact match)

    '''
    verboseprint = print if verbose else lambda *a, **k: None

    sim = {}
    # Calculate the similarity
    for i in range(len(docs)):
      sim[i] = np.dot(df.loc[:, i].values, query_vector) / np.linalg.norm(df.loc[:, i]) * np.linalg.norm(query_vector)
    
    # Sort the values 
    sim_sorted = sorted(sim.items(), key=lambda x: x[1], reverse=True)

    # verboseprint the articles and their similarity values
    perfect_match = 'NaN'
    top_sorted = []
    for k, v in sim_sorted:
        if v >= 0.1:
            top_sorted.append((docs[k][0], docs[k][1], v))

        # Check for 'perfect' (exact) title match
        if v == 1.0:
            perfect_match = docs[k]
    
    return top_sorted, perfect_match
  

In [ ]:
def get_additional_matches(user_query, talk_id, ted_talks_df, complete_list, verbose=False):
    '''
    Count additional feature matches to tiebreak 

    Inputs:
    - user_query : string of user inputted search query
    - ted_talks_df : Pandas Dataframe of ted talks csv
    - top_10_search_results : list of talk ids matching the search query 
    - verbose : whether to print or not (works with verboseprint and verbosepprint)

    Outputs:
    - top_10_search_results : completed search results with additional results

    '''

    verboseprint = print if verbose else lambda *a, **k: None
    verbosepprint = pprint if verbose else lambda *a, **k: None

    total_matches = 0


    # POS tag user query
    light_preprocess = preprocess_no_id(user_query, remove_sw=False, stem=False, untokenized=False, verbose=verbose)
    preprocessed_query = light_preprocess
    tagged = nltk.pos_tag(preprocessed_query)
    verboseprint(tagged)

    # Find any nouns in user query
    query_nouns = []
    for tag in tagged:
        if tag[1] in ['NN', 'NNS', 'NNP', 'NNPS']:
            query_nouns.append(tag[0])

    # Find named entities in search query
    nlp = en_core_web_sm.load()
    named_entities = nlp(user_query)
    verboseprint(named_entities)

    # Get row of current talk id
    talk_row = ted_talks_df.loc[ted_talks_df['talk_id'] == talk_id]

    all_speakers_txt = talk_row['all_speakers'].to_string(index=False)
    all_speakers_dict = ast.literal_eval(all_speakers_txt.strip())
    all_speakers = list(all_speakers_dict.values())
    verboseprint(f"All Speakers: {all_speakers}")

    topics_txt = talk_row['topics'].to_string(index=False)
    topics_list = ast.literal_eval(topics_txt.strip())
    verboseprint(f"Related topics: {topics_list}")

    for entity in named_entities:
        if str(entity) in all_speakers:
            total_matches += 1

    for noun in query_nouns:
        if noun in topics_list:
            total_matches += 1
    
    views = talk_row['views'].to_string(index=False)
    views = int(views)        

    return total_matches, views
            

In [ ]:
# For all talks within a specific range (default +-0.05), determine highest rank based on extra non-tfidf features

def tiebreak(complete_list, user_query, ted_talks_df, range=0.05, verbose=False):

    verboseprint = print if verbose else lambda *a, **k: None

    # sort smallest to largest
    complete_list = sorted(complete_list, key= lambda x: x[1])
    verboseprint(complete_list)

    if len(complete_list) == 0:
        return complete_list
    elif len(complete_list) == 1:
        
        return complete_list
    
    
    max_value = 0
    min_value = complete_list[0][1]
    for id, cosine_val in complete_list:
        if cosine_val > max_value:
            max_value = cosine_val
        elif cosine_val < min_value:
            min_value = cosine_val

    verboseprint(f"Max cosine: {max_value}")
    verboseprint(f"Min cosine: {min_value}")

    bucketed_talks = []
    temp_list = []
    curr_ceiling = min_value + range


    i = 0
    for id, avg_score in complete_list:
        tiebreak_score, views = get_additional_matches(user_query, id, ted_talks_df, complete_list, verbose=False)
        # verboseprint(f"For {id}: Matching features is, {tiebreak_score}, and total views are, {views}")

        # Place last id into appropriate bucket
        if i == (len(complete_list) - 1):
            if avg_score <= curr_ceiling:
                temp_list.append(list((id, avg_score, tiebreak_score, views)))
            else:
                bucketed_talks.append(temp_list)
                temp_list = []
                temp_list.append(list((id, avg_score, tiebreak_score, views)))
            bucketed_talks.append(temp_list)
            verboseprint(f"Complete bucket, final id: {temp_list}")

        # Placing non-final ids
        else:
            if avg_score <= curr_ceiling:
                temp_list.append(list((id, avg_score, tiebreak_score, views)))
            else:
                verboseprint(f"Complete bucket: {temp_list}")
                bucketed_talks.append(temp_list)
                temp_list = []
                temp_list.append(list((id, avg_score, tiebreak_score, views)))
                curr_ceiling = curr_ceiling + range

        i += 1

    # Find talk with most views in bucket
    for talks in bucketed_talks:
        max_views = talks[0][3]
        i = 0
        for talk in talks:
            views = talk[3]
            if views >= max_views:
                highest_viewed_talk = i
            i += 1
        # +1 to talk with most views in bucket
        talks[highest_viewed_talk][2] = talks[highest_viewed_talk][2] + 1
    

    final_sorted_list = []
    for talks in bucketed_talks:
        verboseprint(f"Bucket: {talks}")
        
        sorted_talks = sorted(talks, key=lambda x: x[2] if x[2] > 0 else x[1], reverse=False)

        for talk in sorted_talks:
            final_sorted_list.append((talk[0], talk[1]))

    final_sorted_list.reverse()
    verboseprint(f"Final list: {final_sorted_list}")
    
    return final_sorted_list


def get_average_cosine_score(titles_id_cosine, description_id_cosine, transcripts_id_cosine, top_titles, top_descriptions, top_transcripts, verbose=False):

    verboseprint = print if verbose else lambda *a, **k: None

    complete_list = []
    average_scores = {}

     # Find average score between titles list, descriptions list, and 
    verboseprint("\nFinding matches between top titles list and top descriptions list...")
    boosted_scores = []
    for title, title_talk_id, title_cosine_val in top_titles:
        for description, description_talk_id, description_cosine_val in top_descriptions:
            if title_talk_id == description_talk_id:
                boosted_cosine_val = title_cosine_val + description_cosine_val
                boosted_scores.append((title_talk_id, boosted_cosine_val))
    
    sorted_boosted_scores = sorted(boosted_scores, key=lambda x: x[1], reverse=True)
    title_desc_scores = titles_id_cosine
    title_desc_scores.extend(description_id_cosine)
    title_desc_scores.extend(sorted_boosted_scores)
    sorted_title_desc_matches_scores = sorted(title_desc_scores, key=lambda x: x[1], reverse=True)


    verboseprint("\nCosine scores with added cosine scores for matching talk_ids between titles and descriptions.")
    if len(sorted_title_desc_matches_scores) > 0:
        verboseprint(sorted_title_desc_matches_scores)
    else:
        verboseprint("None found.")

    boosted_scores = []
    for talk_id, cosine_val in sorted_title_desc_matches_scores:
        for transcript, transcript_talk_id, transcript_cosine_val in top_transcripts:
            if talk_id == transcript_talk_id:
                boosted_cosine_val = cosine_val + transcript_cosine_val
                boosted_scores.append((talk_id, boosted_cosine_val))

    complete_returns = sorted_title_desc_matches_scores
    complete_returns.extend(transcripts_id_cosine)
    complete_returns.extend(boosted_scores)
    
    averaged_complete_returns = []
    for id, cosine_total in complete_returns:
        cosine_average = cosine_total / 3
        averaged_complete_returns.append((id, cosine_average))

    sorted_avg_complete_returns = sorted(averaged_complete_returns, key=lambda x: x[1], reverse=True)

    return sorted_avg_complete_returns

def get_top_matches(user_query, ted_talks_df, tfidf_title_model, tfidf_title_df, processed_titles, tfidf_description_model, tfidf_description_df, processed_description, tfidf_transcript_model, tfidf_transcript_df, processed_transcripts, verbose=False):
    '''
    Main driver for processing user query and finding cosine similarity of documents

    Inputs( works with main_tfidf() ):
    - user_query : user inputted query
    - ted_talks_df : Pandas Dataframe of Ted Talks csv
    - tfidf_title_model : tfidf vectorizer model object for Title documents
    - tfidf_title_df : Pandas dataframe of TFIDF vectorized Title tokens (index = title tokens)
    - processed_titles : list of lists with (preprocessed Title text, ted_talk_id)
    - tfidf_description_model : tfidf vectorizer model object for Description documents
    - tfidf_description_df : Pandas dataframe of TFIDF vectorized Description tokens (index = description tokens)
    - processed_description : list of lists with (preprocessed description text, ted_talk_id)
    - tfidf_transcript_model : tfidf vectorizer model object for Transcript documents
    - tfidf_transcript_df : Pandas dataframe of TFIDF vectorized Description tokens (index = transcript tokens)
    - processed_transcripts : list of lists with (preprocessed transcript text, ted_talk_id)
    - verbose : whether to print or not (with verboseprint)

    Outputs:
    - top_10_search_results : list of 10 or less ted talk ids with the greatest cosine similarity scores (sorted highest to lowest)
    - original_results_found : number of results found through tfidf before get_additional_matches() called

    '''
    verboseprint = print if verbose else lambda *a, **k: None

    top_10_search_results = []
    best_title_matches = []
    best_transcript_matches = []  
    
    verboseprint("\nFINDING TOP TITLE MATCHES")
    query_vector = process_query(user_query, tfidf_model=tfidf_title_model, tfidf_df=tfidf_title_df, title_search=True)
    top_titles, perfect_title_match = get_similar_documents(query_vector=query_vector, tfidf_model=tfidf_title_model, df=tfidf_title_df, docs=processed_titles, verbose=verbose)
    verboseprint(f"{len(top_titles)} matches found.")
    if len(top_titles) > 0:
        verboseprint(f"Example: {top_titles[0]}")
    titles_id_cosine = []
    for title, talk_id, cosine_val in top_titles:
        titles_id_cosine.append((talk_id, cosine_val))
    if perfect_title_match != 'NaN':
        verboseprint(f"Perfect match found: {perfect_title_match}")

    verboseprint("\nFINDING TOP DESCRIPTION MATCHES")
    query_vector = process_query(user_query, tfidf_model=tfidf_description_model, tfidf_df=tfidf_description_df, title_search=False)
    top_descriptions, perfect_description_match = get_similar_documents(query_vector=query_vector, tfidf_model=tfidf_description_model, df=tfidf_description_df, docs=processed_description, verbose=verbose)
    verboseprint(f"{len(top_descriptions)} matches found.")
    if len(top_descriptions) > 0:
        verboseprint(f"Example: {top_descriptions[0]}")
    description_id_cosine = []
    for description, talk_id, cosine_val in top_descriptions:
        description_id_cosine.append((talk_id, cosine_val))
    if perfect_description_match != 'NaN':
        verboseprint(f"Perfect match found: {perfect_description_match}")

    verboseprint("\nFINDING TOP TRANSCRIPT MATCHES")
    query_vector = process_query(user_query, tfidf_model=tfidf_transcript_model, tfidf_df=tfidf_transcript_df)
    top_transcripts, perfect_transcript_match = get_similar_documents(query_vector=query_vector, tfidf_model=tfidf_transcript_model, df=tfidf_transcript_df, docs=processed_transcripts, verbose=verbose)
    verboseprint(f"{len(top_transcripts)} matches found.")
    if len(top_transcripts) > 0:
        verboseprint(f"Example: {top_transcripts[0]}")
    transcripts_id_cosine = []
    for transcript, talk_id, cosine_val in top_transcripts:
        transcripts_id_cosine.append((talk_id, cosine_val))
    if perfect_transcript_match != 'NaN':
        verboseprint(f"Perfect match found: {perfect_transcript_match}")

    # Add perfect match to top_10_search_results if found.
    if perfect_title_match != 'NaN':
        top_10_search_results.append(perfect_title_match[1])
    elif perfect_description_match != 'NaN':
        top_10_search_results.append(perfect_description_match[1])
    elif perfect_transcript_match != 'NaN':
        top_10_search_results.append(perfect_transcript_match[1])

    sorted_averages = get_average_cosine_score(titles_id_cosine, description_id_cosine, transcripts_id_cosine, top_titles, top_descriptions, top_transcripts, verbose)
    if len(sorted_averages) > 20:
        sorted_averages = sorted_averages[0:20]

    final_list = tiebreak(sorted_averages, user_query, ted_talks_df, range=0.05)


    verboseprint("\nCosine scores with added cosine scores for matching talk_ids.")
    if len(final_list) > 0:
        verboseprint(final_list)
    else:
        verboseprint("None found.")

    # Add matches to top_10_search_results (a talk would only already be in the top_10_search results if it was a perfect match.)
    for id, cosine, in final_list:
        if id not in top_10_search_results:
            top_10_search_results.append(id)

    # If results greater than 10, only keep top 10.
    num_results_found = len(top_10_search_results)
    original_results_found = num_results_found
    if num_results_found > 10:
        top_10_search_results = top_10_search_results[0:10]
        verboseprint(top_10_search_results)
    
    # If results equals 10, keep all 10.
    elif num_results_found == 10:
        verboseprint(top_10_search_results)
    
    # If results less than 10, look for more results.
    elif 0 < num_results_found < 10:
        top_result = top_10_search_results[0]
        row = ted_talks_df.loc[ted_talks_df['talk_id'] == top_result]
        related_talks = row["related_talks"].to_string(index=False)

        related_dict = ast.literal_eval(related_talks.strip())
        related_talks_list = list(related_dict.keys())

        for related_talk in related_talks_list:
            if related_talk not in top_10_search_results and num_results_found < 10:
                top_10_search_results.append(related_talk)
                num_results_found += 1

    return top_10_search_results, original_results_found

In [ ]:
def get_search_query():
      '''
      Asks user for search query

      Inputs:
      N/A

      Outputs:
      - user_query : string of user inputted query.
      
      '''
      user_query = input("Search:")
      return user_query

In [ ]:
def print_search_results(top_10_search_results, original_results_found, ted_talks_df, verbose=True):
    '''
    Pretty print search results of user search query.

    Inputs:
    - top_10_search_results : list of highest cosine similarity score documents (length 10 or less)
    - original_results_found : number of 'high quality' results found before get_additional_matches()
    - ted_talks_df : Pandas Dataframe of Ted Talks csv
    - verbose: whether to print or not (works with verboseprint)

    Outputs:
    N/A
    '''

    verboseprint = print if verbose else lambda *a, **k: None

    alternates = False
    if len(top_10_search_results) == 0:
        verboseprint(colored(f"No results found. Please search for something else...", 'red'))
        true_results = 0
    else:
        verboseprint(colored(f"{original_results_found} result(s) found. Top 10 returned:", 'green'))

    search_ranking = 1
    for id in top_10_search_results[0:original_results_found]:
        row = ted_talks_df.loc[ted_talks_df['talk_id'] == id]
        title = row['title'].to_string(index=False)
        main_speaker = row["speaker_1"].to_string(index=False)
        event = row["event"].to_string(index=False)
        description = row["description"].to_string(index=False)
        url = row['url'].to_string(index=False)
        verboseprint(colored(f"{search_ranking}: {title}", 'blue', attrs=['bold']))
        verboseprint(colored("    Presented by" + main_speaker + " at" + event, 'blue'))
        verboseprint("   " + description)
        verboseprint(colored("   " + url, 'yellow'))
        verboseprint("-----------------------------------------------\n")
        search_ranking = search_ranking + 1

    if len(top_10_search_results) - original_results_found > 0:
        verboseprint(colored(f"End of results.", 'red'))
        verboseprint(colored(f"Based on top result, here are other talks you may be interested in...", 'yellow'))
        for id in top_10_search_results[original_results_found:]:
            row = ted_talks_df.loc[ted_talks_df['talk_id'] == id]
            title = row['title'].to_string(index=False)
            main_speaker = row["speaker_1"].to_string(index=False)
            event = row["event"].to_string(index=False)
            description = row["description"].to_string(index=False)
            verboseprint(colored(f"{search_ranking}: {title}", 'blue', attrs=['bold']))
            verboseprint(colored("    Presented by" + main_speaker + " at" + event, 'blue'))
            verboseprint("   " + description)
            verboseprint("-----------------------------------------------\n")
            search_ranking = search_ranking + 1
    
    verboseprint(colored(f"End of results.", 'red'))

In [ ]:
# PRE SEARCHING FUNCTIONS (DO ONCE)
filepath = '/content/drive/MyDrive/ted_talks_en.csv'
ted_talks_df = pd.read_csv(filepath)
tfidf_title_model, tfidf_title_df, processed_titles, tfidf_description_model, tfidf_description_df, processed_description, tfidf_transcript_model, tfidf_transcript_df, processed_transcripts = main_tfidf(ted_talks_df, verbose=True)

In [ ]:
user_query = get_search_query()
top_10_search_results, original_results_found = get_top_matches(user_query, ted_talks_df, tfidf_title_model, tfidf_title_df, processed_titles, tfidf_description_model, tfidf_description_df, processed_description, tfidf_transcript_model, tfidf_transcript_df, processed_transcripts, verbose=False)
print_search_results(top_10_search_results, original_results_found, ted_talks_df, verbose=True)

Search:crack
8 result(s) found. Top 10 returned:
1:  What if cracks in concrete could fix themselves?
    Presented by Congrui Jin at TED-Ed
    Concrete is the most widely used construction material in the world. It can be found in swathes of city pavements, bridges that span vast rivers and the tallest skyscrapers on earth. But it does have a weakness: it's prone to catastrophic cracking that has immense financial and environmental impact. What if we could avoid that problem? Congrui Jin explores how to create a more resilient concrete. [TED-Ed Animation by Aeon Production].
    https://www.ted.com/talks/congrui_jin_what_if_cracks_in_concrete_could_fix_themselves/
-----------------------------------------------

2:  The freakonomics of crack dealing
    Presented by Steven Levitt at TED2004
    "Freakonomics" author Steven Levitt presents new data on the finances of drug dealing. Contrary to popular myth, he says, being a street-corner crack dealer isn't lucrative: It pays below mini